In [ ]:
from ultralytics import YOLO
import os
import pytesseract
import cv2
import numpy

In [ ]:
# Load the model
model = YOLO("/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/receipt_server/model/versions/V0_2/receipt_extractor.pt")

In [ ]:
image_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/test/images/0676aa86-IMG_20231221_163400_jpg.rf.3ef3b7a564323dc7d96c82e257c4c400.jpg"
annotation_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/test/labels/0676aa86-IMG_20231221_163400_jpg.rf.3ef3b7a564323dc7d96c82e257c4c400.txt"

In [ ]:
def get_highest_prediction(model, image_path)-> dict:
    # Get the predictions
    predictions = model(image_path)

    # Get the highest prediction
    highest_conf_detections = {}
    for result in predictions:
        # print(result)
        for box in result.boxes:
            class_id = int(box.cls)
            class_label = result.names[int(box.cls)]
            conf = box.conf

            # Check if the class label is already in the dictionary
            if class_label in highest_conf_detections:
                # Check if the current confidence is higher than the one stored
                if conf > highest_conf_detections[class_label]["conf"]:
                    highest_conf_detections[class_label] = {
                    "conf": conf,
                    "class_label": class_label,
                    "class_id": class_id,
                    "coordinates": {
                        "x1": int(box.xyxy[0][0]),
                        "y1": int(box.xyxy[0][1]),
                        "x2": int(box.xyxy[0][2]),
                        "y2": int(box.xyxy[0][3]),
                    }
                }
            else:
                highest_conf_detections[class_label] = {
                    "conf": conf,
                    "class_label": class_label,
                    "class_id": class_id,
                    "coordinates": {
                        "x1": int(box.xyxy[0][0]),
                        "y1": int(box.xyxy[0][1]),
                        "x2": int(box.xyxy[0][2]),
                        "y2": int(box.xyxy[0][3]),
                    }
                }


    return highest_conf_detections

extracted_prediction = get_highest_prediction(model, image_path)
print(extracted_prediction)

In [ ]:

def convert_yolo_coordinates(file_path, img_width, img_height)->dict:

    class_mapping = {
        0: "date",
        1: "item_purchase",
        2: "shop_information",
        3: "total",
    }
        
    labels_coordinates = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue  # Skip invalid lines

        class_id, x_center, y_center, width, height = map(float, parts)

        # Calculate the coordinates of the bounding box (x1, y1, x2, y2)
        x1 = int((x_center - width / 2) * img_width)
        y1 = int((y_center - height / 2) * img_height)
        x2 = int((x_center + width / 2) * img_width)
        y2 = int((y_center + height / 2) * img_height)

        labels_coordinates[class_mapping[int(class_id)]] = {
            "x1": x1,
            "y1": y1,
            "x2": x2,
            "y2": y2
        }
    return labels_coordinates

convert_yolo_coordinates(annotation_path, 640, 640)


In [ ]:
def calculate_accuracy(model, image_path, annotation_path, image_name):
    predictions = get_highest_prediction(model, image_path)
    labels = convert_yolo_coordinates(annotation_path, 640, 640)
    accuracies = {}
    is_detected = {}

    # Initialize all classes with None
    for class_name in labels.keys():
        accuracies[class_name] = None

    #for each prediction, display the bounding box on the image and save it
    # print(f"label")
    for key in labels:

        is_detected[key] = True

        if key not in predictions:
            print(f"{key} not found")
            accuracies[key] = None
            continue

        #load the image
        image = cv2.imread(image_path)

        # print(f"{key} : {predictions[key]['class_id']}")

        real_coordinates = labels[key]
        predicted_coordinates = predictions[key]["coordinates"]
        
        # Draw the predicted bounding box with name
        x1 = predicted_coordinates["x1"]
        y1 = predicted_coordinates["y1"]
        x2 = predicted_coordinates["x2"]
        y2 = predicted_coordinates["y2"]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, "prediction", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Draw the real bounding box with name
        x1 = real_coordinates["x1"]
        y1 = real_coordinates["y1"]
        x2 = real_coordinates["x2"]
        y2 = real_coordinates["y2"]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(image, "real", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Calculate the common area between the two bounding boxes if they intersect
        x1 = max(real_coordinates["x1"], predicted_coordinates["x1"])
        y1 = max(real_coordinates["y1"], predicted_coordinates["y1"])
        x2 = min(real_coordinates["x2"], predicted_coordinates["x2"])
        y2 = min(real_coordinates["y2"], predicted_coordinates["y2"])
        if x1 >= x2 or y1 >= y2:
            # print(f"{key} not intersecting")
            accuracies[key] = 0
            is_detected[key] = False
            # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
            # cv2.putText(image, " not intersection", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
            accuracies[key] = 0
            continue
        # else:
        #     cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
        #     cv2.putText(image, "intersection", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        #save picture
        # print(f"saving prediction for {image_name} {key}")
        # print(f"at /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/notebook/evaluation_data/prediction/{image_name}/{key}_prediction.png")
        if not os.path.exists(f"./evaluation_data/temp/{image_name.replace('.','_')}"):
            os.makedirs(f"./evaluation_data/temp/{image_name.replace('.','_')}")
        cv2.imwrite(f"./evaluation_data/temp/{image_name.replace('.','_')}/{key}_prediction.png", image)

        #calculate the area of the union over the area of the label
        area_intersection = (x2 - x1) * (y2 - y1)
        area_label = (real_coordinates["x2"] - real_coordinates["x1"]) * (real_coordinates["y2"] - real_coordinates["y1"])
        accuracy = area_intersection / area_label
        accuracies[key] = accuracy

    return accuracies
# calculate_accuracy(model, image_path, annotation_path, "test")

In [ ]:
image_folder_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/notebook/evaluation_data/images/"
annotation_folder_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/notebook/evaluation_data/labels/"

model_paths = {
   0.3: "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/receipt_server/model/versions/V0_3/receipt_extractor.pt",
}

for version, model_path in model_paths.items():
    model = YOLO(model_path)
    #list all the images in the folder wihout the extension
    images = [os.path.splitext(file)[0] for file in os.listdir(image_folder_path) if file.endswith(".jpg")]
    all_accuracy = {}
    for image in images:
        image_path = os.path.join(image_folder_path, image + ".jpg")
        annotation_path = os.path.join(annotation_folder_path, image + ".txt")
        accuracies = calculate_accuracy(model, image_path, annotation_path, image)
        all_accuracy[image] = accuracies                     

    # for each class calculate the average accuracy and the number of None
    average_accuracy = {}
    for key in ["date", "item_purchase", "shop_information", "total"]:
        average_accuracy[key] = {
            "average_accuracy": 0,
            "number_of_none": 0
        }
        for image in images:
            if all_accuracy[image][key] is not None:
                average_accuracy[key]["average_accuracy"] += all_accuracy[image][key]
            else:
                average_accuracy[key]["number_of_none"] += 1
        average_accuracy[key]["average_accuracy"] = average_accuracy[key]["average_accuracy"] / (len(images) - average_accuracy[key]["number_of_none"])

    print("\n########################################################################\nAccuracy for version", version, ":")
    print(average_accuracy)

    overall_accuracy = 0
    for key in average_accuracy:
        overall_accuracy += average_accuracy[key]["average_accuracy"]
    overall_accuracy = overall_accuracy / len(average_accuracy)

    print(f"Overall accuracy: {overall_accuracy}")
